In [15]:
import pandas as pd
import os
from scipy.special import boxcox1p
import seaborn as sns

In [2]:
# build data directory
data_dir = '../../data/stage1'
os.listdir(data_dir)

['Churn_OD.txt',
 'Churn_UsersProfile.txt',
 'stage_1_challenge_1_dictionary.xlsx']

In [3]:
# Load user profiles
user_profiles_df = pd.read_csv(
    os.path.join(data_dir, 'Churn_UsersProfile.txt'), 
    delimiter = "|",
    encoding = "ISO-8859-1"
)

## Filter for the two periods - do we need to normalise for the extra month on p1? - how?
### P2 during pandemic?

In [6]:
period1 = (user_profiles_df.loc[user_profiles_df['Period'] == 'Sep-19 to Feb-20']
             .rename(columns={'Average_BusUsers_per_Day': 'Average_BusUsers_per_Day_p1'})
             .drop(columns = ['Period']))
period2 = (user_profiles_df.loc[user_profiles_df['Period'] == 'Sep-20 to Jan-21']
             .rename(columns={'Average_BusUsers_per_Day': 'Average_BusUsers_per_Day_p2'})
             .drop(columns = ['Period']))
period2.head(1)

,Region_of_Origin,District_of_Origin,County_of_Origin,GenderDescription,AgeClassDescription,Average_BusUsers_per_Day_p2
255,R1 - AM Lisboa,Lisboa,Amadora,Female,15-24,536.005174


 ## Join the two period tables to get a side by side view of Average Bus users per day per user profile

In [7]:
df_merged = pd.merge(period1, period2, on=['Region_of_Origin', 'District_of_Origin', 'County_of_Origin',
                                           'GenderDescription', 'AgeClassDescription'])

In [8]:
df_merged.head(5)

,Region_of_Origin,District_of_Origin,County_of_Origin,GenderDescription,AgeClassDescription,Average_BusUsers_per_Day_p1,Average_BusUsers_per_Day_p2
0,R1 - AM Lisboa,Lisboa,Amadora,Female,15-24,294.194206,536.005174
1,R1 - AM Lisboa,Lisboa,Amadora,Female,25-34,1081.652817,408.068335
2,R1 - AM Lisboa,Lisboa,Amadora,Female,35-44,235.836653,478.337192
3,R1 - AM Lisboa,Lisboa,Amadora,Female,45-54,840.951323,571.192813
4,R1 - AM Lisboa,Lisboa,Amadora,Female,55-64,1216.148092,741.072367


In [9]:
df_merged['Churn'] = df_merged['Average_BusUsers_per_Day_p2'] - df_merged['Average_BusUsers_per_Day_p1']
df_merged['Churn_percent'] = ((df_merged['Average_BusUsers_per_Day_p2'] - df_merged['Average_BusUsers_per_Day_p1'])/ df_merged['Average_BusUsers_per_Day_p1'])*100

In [10]:
df_merged.sort_values(by=['Churn_percent'], ascending = False).head(10)

,Region_of_Origin,District_of_Origin,County_of_Origin,GenderDescription,AgeClassDescription,Average_BusUsers_per_Day_p1,Average_BusUsers_per_Day_p2,Churn,Churn_percent
139,R1 - AM Lisboa,Setúbal,Seixal,Male,15-24,37.697725,1631.340763,1593.643038,4227.424965
103,R1 - AM Lisboa,Setúbal,Almada,Male,25-34,17.528597,497.588139,480.059542,2738.721998
63,R1 - AM Lisboa,Lisboa,Oeiras,Female,25-34,21.863276,447.437221,425.573945,1946.524139
238,R2 - AM Porto,Porto,Vila Nova de Gaia,Male,55-64,21.943108,374.627538,352.684430,1607.267439
223,R2 - AM Porto,Porto,Valongo,Male,25-34,27.625672,366.664832,339.039160,1227.261223
184,R2 - AM Porto,Porto,Maia,Female,55-64,49.697015,414.680964,364.983949,734.418252
7,R1 - AM Lisboa,Lisboa,Amadora,Male,35-44,137.697607,723.561851,585.864244,425.471623
94,R1 - AM Lisboa,Lisboa,Vila Franca de Xira,Male,55-64,141.556626,693.873061,552.316435,390.173495
121,R1 - AM Lisboa,Setúbal,Montijo,Female,35-44,65.504612,317.566534,252.061922,384.800267
39,R1 - AM Lisboa,Lisboa,Loures,Female,65+,147.474840,702.814481,555.339641,376.565685


## Analyse transport use changes for each age group

In [11]:
df_age_comp = (df_merged.groupby(['AgeClassDescription'])
                        .agg({'Average_BusUsers_per_Day_p1': 'sum',
                              'Average_BusUsers_per_Day_p2': 'sum'}))

df_age_comp['Churn'] = df_age_comp['Average_BusUsers_per_Day_p2'] - df_age_comp['Average_BusUsers_per_Day_p1']
df_age_comp['Churn_percent'] = ((df_age_comp['Average_BusUsers_per_Day_p2'] - df_age_comp['Average_BusUsers_per_Day_p1'])/ df_age_comp['Average_BusUsers_per_Day_p1'])*100
df_age_comp.head(10)

,Average_BusUsers_per_Day_p1,Average_BusUsers_per_Day_p2,Churn,Churn_percent
AgeClassDescription,,,,
15-24,38287.457553,21844.347320,-16443.110233,-42.946467
25-34,32792.788327,15571.847812,-17220.940515,-52.514414
35-44,29652.700723,20061.064491,-9591.636232,-32.346586
45-54,45707.615627,27429.148887,-18278.466740,-39.989981
55-64,51675.402952,32439.272948,-19236.130004,-37.224925
65+,191713.975969,107189.407724,-84524.568245,-44.088892


## Analyse transport use changes per Region


In [13]:
df_loc_comp = (df_merged.groupby(['Region_of_Origin', 'District_of_Origin','County_of_Origin'])
                        .agg({'Average_BusUsers_per_Day_p1': 'sum',
                              'Average_BusUsers_per_Day_p2': 'sum'}))

df_loc_comp['Churn'] = df_loc_comp['Average_BusUsers_per_Day_p2'] - df_loc_comp['Average_BusUsers_per_Day_p1']
df_loc_comp['Churn_percent'] = ((df_loc_comp['Average_BusUsers_per_Day_p2'] - df_loc_comp['Average_BusUsers_per_Day_p1'])/ df_loc_comp['Average_BusUsers_per_Day_p1'])*100
#df_loc_comp.head(100)

In [14]:
df_loc_comp_churn = (df_merged.groupby(['Region_of_Origin', 'District_of_Origin','County_of_Origin'])
                        .agg({'Churn_percent': 'mean'}))
#df_loc_comp_churn.head(100)